In [1]:
import os

from bids import BIDSLayout

from glob import glob

from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function

In [2]:
bids_dir = os.path.join('/fetaldata')
output_dir = os.path.join('/fetaldata','derivatives','mialsrtk')
mask_dir = os.path.join('/fetaldata','derivatives','manual_masks')

subject = '01'

layout = BIDSLayout(bids_dir,derivatives=mask_dir)
print(layout)

BIDS Layout: .../fetaldata | Subjects: 1 | Sessions: 2 | Runs: 6


In [3]:
import subprocess
def run(self, command, env={}, cwd=os.getcwd()):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.run(command, shell=True,
                               env=merged_env, cwd=cwd, capture_output=True)
    return process

In [4]:
from traits.api import *

import nibabel as nib

from nipype.utils.filemanip import split_filename
from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

from nipype.interfaces.mixins import reporting



In [5]:
class MialsrtkImageReconstructionInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    
    
    in_roi = traits.Enum(None, "all", "box", "mask", mandatory = True)
    in_deblurring = traits.Bool(False, usedefault=True)
    in_reg = traits.Bool(True, usedefault=True)
    in_3d = traits.Bool(False, usedefault=True)
    
    in_margin = traits.Float(usedefault=False)
    in_epsilon = traits.Float(usedefault=False)
    in_iter = traits.Int(usedefault=False)
    
    in_combinedMasks = traits.Str(usedefault=False) ## ?? TODO
    in_reference = File(desc='Reference image') # , mandatory=True)
    out_postfix = traits.Str("_sr", usedefault=True)
    
    
    in_imresampled = InputMultiPath(File(desc='')) # , mandatory = True))
    in_imroi = InputMultiPath(File(desc='')) # , mandatory = True))
    in_transform = InputMultiPath(File(desc='')) # , mandatory = True))
    in_masks = InputMultiPath(File(desc='')) # , mandatory = True))
    input_images = InputMultiPath(File(desc='')) # , mandatory = True))
            
    
class MialsrtkImageReconstructionOutputSpec(TraitedSpec):
    output_images = OutputMultiPath(File())

class MialsrtkImageReconstruction(BaseInterface):
    input_spec = MialsrtkImageReconstructionInputSpec
    output_spec = MialsrtkImageReconstructionOutputSpec

    def _run_interface(self, runtime):
        
        print("input_image", self.inputs.input_images)

        params = []

        params.append(''.join(["--", self.inputs.in_roi]))
        if self.inputs.in_deblurring:
            params.append("--deblurring")

        if not self.inputs.in_reg:
            params.append("--noreg")

        if self.inputs.in_3d:
            params.append("--3D")

        if self.inputs.in_margin:
            params.append("--margin")
            params.append(str(self.inputs.in_margin))

        if self.inputs.in_epsilon:
            params.append("--epsilon")
            params.append(str(self.inputs.in_epsilon))

        if self.inputs.in_iter:
            params.append("--iter")
            params.append(str(self.inputs.in_iter))

        if self.inputs.in_combinedMasks:
            params.append("--combinedMasks")
            params.append(str(self.inputs.in_combinedMasks))

        if self.inputs.in_reference:
            params.append("--reference")
            params.append(str(self.inputs.in_reference))

        _, name, ext = split_filename(os.path.abspath(self.inputs.input_images[0]))
        out_file = os.path.join(os.getcwd().replace(self.inputs.bids_dir,'/fetaldata'), ''.join((name, self.inputs.out_postfix, ext)))
        params.append("--output")
        params.append(out_file)


        if self.inputs.in_imresampled:
            for ir in self.inputs.in_imresampled:
                params.append("--ir")
                params.append(ir)

        if self.inputs.in_imroi:
            for roi in self.inputs.in_imroi:
                params.append("--roi")
                params.append(roi)

        if self.inputs.in_transform:
            for t in self.inputs.in_transform:
                params.append("--transform")
                params.append(t)

        if self.inputs.in_masks:
            for m in self.inputs.in_masks:
                params.append("-m")
                params.append(m)

        if self.inputs.input_images:
            for i in self.inputs.input_images:
                params.append("--input")
                params.append(i)

        
        cmd = ["mialsrtkImageReconstruction"] 
        cmd += params
        
#         cmd = ["mialsrtkImageReconstruction", "--help"]
        
        try:
            print('... cmd: {}'.format(cmd))
            cmd = ' '.join(cmd)
            run(self, cmd, env={}, cwd=os.path.abspath(self.inputs.bids_dir))
        except:
            print('Failed')
        return runtime
            
        
    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['output_images'] = glob(os.path.abspath("*.nii.gz"))
        return outputs

In [6]:
#Check if mandatory derivatives dataset_description.json exists in derivatives/mialsrtk.
# If not, it is created before running the workflow, otherwise BIDSDataGrabber is not happy and raises an error. 

mialsrtk_dataset_description_json = os.path.join(output_dir,'dataset_description.json')

print('Check for {}'.format(mialsrtk_dataset_description_json))
if not os.access(mialsrtk_dataset_description_json, os.R_OK):
    import json
    data = {'PipelineDescription':{'Name': 'MIAL Super-Resolution ToolKit', 
                                'Version': 'v2.0.0-beta', 
                                'CodeURL': 'https://github.com/sebastientourbier/mialsuperresolutiontoolkit'
                                  },
            'Name': 'MIAL Super-Resolution ToolKit',
            'BIDSVersion': '1.2.0'
           }
    os.makedirs(output_dir)
    with open(mialsrtk_dataset_description_json, 'w+') as outfile:
        json.dump(data, outfile, indent=4)
    print('File {} was created'.format(mialsrtk_dataset_description_json))
else:
    print('File {} already exists'.format(mialsrtk_dataset_description_json))
    

wf = Workflow(name="sr",base_dir=os.path.join(output_dir,'sub-{}'.format(subject),'nipype'))

bg = Node(interface=BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.index_derivatives = True
bg.inputs.raise_on_empty = False
bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"])}


sr = Node(interface=MialsrtkImageReconstruction(),name='_sr')
sr.inputs.bids_dir = bids_dir
sr.inputs.in_roi = "all"
sr.inputs.in_deblurring = True
sr.inputs.in_reg = False
sr.inputs.in_3d = True
sr.inputs.in_margin = 2.15
sr.inputs.in_epsilon = 56.15
sr.inputs.in_iter = 56

sr.inputs.in_combinedMasks = "combined mask" 
sr.inputs.in_reference = "path to ref"
out_postfix = traits.Str("_sr", usedefault=True)

                  
wf.connect(bg, "T2ws", sr, "input_images")
wf.connect(bg, "T2ws", sr, "in_imresampled")
wf.connect(bg, "T2ws", sr, "in_imroi")
wf.connect(bg, "T2ws", sr, "in_transform")
wf.connect(bg, "T2ws", sr, "in_masks")

res = wf.run()

Check for /fetaldata/derivatives/mialsrtk/dataset_description.json
File /fetaldata/derivatives/mialsrtk/dataset_description.json already exists
191017-14:21:54,216 nipype.workflow INFO:
	 Workflow sr settings: ['check', 'execution', 'logging', 'monitoring']
191017-14:21:54,219 nipype.workflow INFO:
	 Running serially.
191017-14:21:54,219 nipype.workflow INFO:
	 [Node] Setting-up "sr.bids_grabber" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/sr/bids_grabber".
191017-14:21:54,222 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")


/opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/bids/layout/layout.py:659: UserWarning: In pybids 0.9.0, the 'extensions' filter was deprecated in favor of 'extension'. The former will stop working in 0.11.0.
  warnings.warn("In pybids 0.9.0, the 'extensions' filter was "


191017-14:21:55,73 nipype.workflow INFO:
	 [Node] Finished "sr.bids_grabber".
191017-14:21:55,73 nipype.workflow INFO:
	 [Node] Setting-up "sr._sr" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/sr/_sr".
191017-14:21:55,78 nipype.workflow INFO:
	 [Node] Cached "sr._sr" - collecting precomputed outputs
191017-14:21:55,78 nipype.workflow INFO:
	 [Node] "sr._sr" found cached.


___

___